In [1]:
from openai import AzureOpenAI
import json
import os
import time

from dotenv import load_dotenv
load_dotenv()

azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT")
azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_KEY")

# Create a client
client = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_endpoint=azure_endpoint,
    api_key=api_key
)

In [2]:
# Create a vector store
vector_store = client.beta.vector_stores.create(name="Contoso Hiking Products")

# Ready the files for upload to OpenAI
file_paths = ["./data/contoso/product_info_1.md", 
              "./data/contoso/product_info_2.md",
              "./data/contoso/product_info_3.md",
              "./data/contoso/product_info_4.md",
              "./data/contoso/product_info_5.md",
              "./data/contoso/product_info_6.md",
              "./data/contoso/product_info_7.md",
              "./data/contoso/product_info_8.md",
              "./data/contoso/product_info_9.md",
              "./data/contoso/product_info_10.md",
              "./data/contoso/product_info_11.md",
              "./data/contoso/product_info_12.md",
              "./data/contoso/product_info_13.md",
              "./data/contoso/product_info_14.md",
              "./data/contoso/product_info_15.md",
              "./data/contoso/product_info_16.md",
              "./data/contoso/product_info_17.md",
              "./data/contoso/product_info_18.md",
              "./data/contoso/product_info_19.md",
              "./data/contoso/product_info_20.md",
              ]
file_streams = [open(path, "rb") for path in file_paths]

# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)

In [3]:
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)
print(vector_store.id)


completed
FileCounts(cancelled=0, completed=20, failed=0, in_progress=0, total=20)
vs_U6FGmPa5rJRo5Iavv88ZWRin


In [4]:
# Create assistant
assistant = client.beta.assistants.create(
  name="Contoso Travel Assistant",
  instructions="""
  You are a Travel assistant that provides information. 
   When asked for products, you will provide recommendations using the product_info_XX.md files provided to you. 
   You will not provide recommendations outside of those product files 
  """,
  model=azure_deployment,
  tools=[{"type":"file_search"}],
  tool_resources={"file_search":{"vector_store_ids":[vector_store.id]}},
  temperature=1,
  top_p=1
)

# Create thread
thread = client.beta.threads.create()
print(thread)



Thread(id='thread_fLUeV1AewE5sM6edqrCg0D5v', created_at=1731715529, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [9]:
#user_question ="""What is the price of the Trailmaster X4 Tent and its capacity?"""

user_question ="""How much is the Trailmaster X4 Tent and the Adventure Dining Table?"""

# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content=user_question
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

  if run.status == 'completed':
    messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  
  elif run.status == 'requires_action':
    pass
  
  else:
    print(run.status)


# Print the assistant response
content_block = messages.data[0].content[0]
value = content_block.text.value
print(value)

The Trailmaster X4 Tent is priced at $250 and the Adventure Dining Table is priced at $90【4:1†source】【4:2†source】.


# Delete Assistant

In [10]:
response = client.beta.assistants.delete(assistant.id)